In [1]:
import pandas as pd 
import numpy as np

df = pd.read_csv('employee_retention.csv')
df.head()

,employee_id,company_id,dept,seniority,salary,join_date,quit_date
0,13021.0,7,customer_service,28,89000.0,2014-03-24,2015-10-30
1,825355.0,7,marketing,20,183000.0,2013-04-29,2014-04-04
2,927315.0,4,marketing,14,101000.0,2014-10-13,NaN
3,662910.0,7,customer_service,20,115000.0,2012-05-14,2013-06-07
4,256971.0,2,data_science,23,276000.0,2011-10-17,2014-08-22


In [3]:
df.dtypes

employee_id    float64
company_id       int64
dept            object
seniority        int64
salary         float64
join_date       object
quit_date       object
dtype: object

In [4]:
df.shape

(24702, 7)

In [6]:
# first task: get the headcount on each day for all companies

from datetime import datetime

df['join_date'] = pd.to_datetime(df['join_date'])
df['quit_date'] = pd.to_datetime(df['quit_date'])

df.describe(include='all')

<ipython-input-6-c2ff61ef35f3>:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')
<ipython-input-6-c2ff61ef35f3>:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe(include='all')


,employee_id,company_id,dept,seniority,salary,join_date,quit_date
count,24702.000000,24702.000000,24702,24702.000000,24702.000000,24702,13510
unique,NaN,NaN,6,NaN,NaN,995,664
top,NaN,NaN,customer_service,NaN,NaN,2012-01-03 00:00:00,2015-05-08 00:00:00
freq,NaN,NaN,9180,NaN,NaN,105,111
first,NaN,NaN,NaN,NaN,NaN,2011-01-24 00:00:00,2011-10-13 00:00:00
last,NaN,NaN,NaN,NaN,NaN,2015-12-10 00:00:00,2015-12-09 00:00:00
mean,501604.403530,3.426969,NaN,14.127803,138183.345478,NaN,NaN
std,288909.026101,2.700011,NaN,8.089520,76058.184573,NaN,NaN
min,36.000000,1.000000,NaN,1.000000,17000.000000,NaN,NaN
25%,250133.750000,1.000000,NaN,7.000000,79000.000000,NaN,NaN


In [16]:
# the list of all days
uniq_dates = pd.date_range(start='2011/01/24', end='2015/12/13')

# the list of all companies
uniq_companies = df['company_id'].unique()

# all possbile combinations of company and date
df_headcount = pd.MultiIndex.from_product([uniq_dates, uniq_companies], names=['date', 'company_id'])
df_headcount = pd.DataFrame(index=df_headcount).reset_index()


# method 1: use join
df_join = df.groupby(['join_date', 'company_id']).size().reset_index(name='join_count')
df_join.columns.values[0] = 'date'

df_quit = df.groupby(['quit_date', 'company_id']).size().reset_index(name='quit_count')
df_quit.columns.values[0] = 'date'

# now join with all possible combinations
df_headcount = pd.merge(df_headcount, df_join, on=['date', 'company_id'], how='left')
df_headcount = pd.merge(df_headcount, df_quit, on=['date', 'company_id'], how='left')

df_headcount.fillna(0, inplace=True)


# do the cumsum
df_headcount['cumsum_join'] = df_headcount.groupby(['company_id'])['join_count'].apply(lambda x:x.cumsum())
df_headcount['cumsum_quit'] = df_headcount.groupby(['company_id'])['quit_count'].apply(lambda x:x.cumsum())

# on each day, get the subtraction of join_count and quit_count
df_headcount['count'] = df_headcount['cumsum_join'] - df_headcount['cumsum_quit']
df_headcount_table = df_headcount[['date', 'company_id', 'count']]

print(df_headcount_table.query('company_id==1').head(5))

         date  company_id  count
4  2011-01-24           1   25.0
16 2011-01-25           1   27.0
28 2011-01-26           1   29.0
40 2011-01-27           1   29.0
52 2011-01-28           1   29.0
